<a href="https://colab.research.google.com/github/CiaraFarrellSETU/Dissertation_code/blob/main/VGG16_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Image Classification Using Transfer Learning (VGG-16)  
https://muhammadrizwanmunawar.medium.com/image-classification-using-transfer-learning-vgg-16-2dc2221be34c

mounting google drvie

In [58]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


importing libaries

In [59]:
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

unzipping zipped file from google drive

In [60]:
import zipfile

with zipfile.ZipFile('/content/gdrive/MyDrive/dog_eyes.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/unzipp_data')  # Replace with desired destination directory

set the path of training, testing, and validation directories

In [61]:
base_dir = '/content/unzipp_data/dog_eyes'

train_dir = '/content/unzipp_data/dog_eyes/train'
cat_train_dir = '/content/unzipp_data/dog_eyes/train/cataract'
healthy_train_dir = '/content/unzipp_data/dog_eyes/train/healthy'

test_dir = '/content/unzipp_data/dog_eyes/test'
cat_test_dir = '/content/unzipp_data/dog_eyes/test/cataract'
healthy_test_dir = '/content/unzipp_data/dog_eyes/test/healthy'

val_dir = '/content/unzipp_data/dog_eyes/valid'
cat_val_dir = '/content/unzipp_data/dog_eyes/valid/cataract'
healthy_val_dir = '/content/unzipp_data/dog_eyes/valid/healthy'

getting data from these folders

In [62]:
num_cat_train = len(os.listdir(cat_train_dir))
num_healthy_train = len(os.listdir(healthy_train_dir))

num_cat_test = len(os.listdir(cat_test_dir))
num_healthy_test = len(os.listdir(healthy_test_dir))

num_cat_val = len(os.listdir(cat_val_dir))
num_healthy_val = len(os.listdir(healthy_val_dir))

seeing how many training,testing and validation images there is

In [63]:
print("Total Training cat Images",num_cat_train)
print("Total Training healthy Images",num_healthy_train)
print("--")
print("Total test cat Images",num_cat_test)
print("Total test healthy Images",num_healthy_test)
print("--")
print("Total validation cat Images",num_cat_val)
print("Total validation healthy Images",num_healthy_val)
print("--")


total_train = num_cat_train+num_healthy_train
total_validation = num_cat_val+num_healthy_val
total_test = num_cat_test+num_healthy_test
print("Total Training Images",total_train)
print("--")
print("Total Validation Images",total_validation)
print("--")
print("Total Testing Images",total_test)

Total Training cat Images 5290
Total Training healthy Images 1530
--
Total test cat Images 179
Total test healthy Images 70
--
Total validation cat Images 421
Total validation healthy Images 121
--
Total Training Images 6820
--
Total Validation Images 542
--
Total Testing Images 249


resize images to (224,224) because VGG-16 only accepts that image size.

In [64]:
IMG_SHAPE  = 224
batch_size = 32

preprocess data (train, test, validation), which includes, rescaling and shuffling

In [65]:
image_gen_train = ImageDataGenerator(rescale = 1./255)

In [66]:
train_data_gen = image_gen_train.flow_from_directory(batch_size = batch_size,
directory = train_dir,
shuffle= True,
target_size = (IMG_SHAPE,IMG_SHAPE),
class_mode = 'binary')
image_generator_validation = ImageDataGenerator(rescale=1./255)

Found 6820 images belonging to 2 classes.


In [67]:
val_data_gen = image_generator_validation.flow_from_directory(batch_size=batch_size,
directory=val_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='binary')
image_gen_test = ImageDataGenerator(rescale=1./255)

Found 541 images belonging to 2 classes.


In [68]:
test_data_gen = image_gen_test.flow_from_directory(batch_size=batch_size,
directory=test_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='binary')

Found 249 images belonging to 2 classes.



The image data generator is supposed to use the folder names as class names.  
folder names are cataract, healthy

In [69]:
pre_trained_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [70]:
for layer in pre_trained_model.layers:
  print(layer.name)
layer.trainable = False

input_3
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [71]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(2, activation='sigmoid')(x)

have tried different activations softmax and sigmoid

In [72]:
model = tf.keras.Model(pre_trained_model.input, x)

In [73]:
#model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['acc'])

In [74]:
model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])

In [75]:
model.summary

<bound method Model.summary of <keras.src.engine.functional.Functional object at 0x7af9098a3dc0>>

In [76]:
model

In [77]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [78]:
vgg_classifier = model.fit(train_data_gen,
steps_per_epoch=(total_train//batch_size),
epochs = 5,
validation_data=val_data_gen,
validation_steps=(total_validation//batch_size),
batch_size = batch_size,
verbose = 1)

Epoch 1/5


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    

In [79]:
print(train_data_gen[0][0].shape, train_data_gen[0][1].shape)

(32, 224, 224, 3) (32,)


In [80]:
print(val_data_gen[0][0].shape, val_data_gen[0][1].shape)

(32, 224, 224, 3) (32,)


train and val are the same shape so unsure why error is occuring

In [ ]:
result = model.evaluate(test_data_gen,batch_size=batch_size)
print("test_loss, test accuracy",result)